In [2]:
import jieba
import numpy as np
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [3]:
from os import listdir
from os.path import isfile, join,splitext
t_dir = 'dataTrainComplete'
txt_fnames = [splitext(f)[0] for f in listdir(t_dir) if isfile(join(t_dir, f))] #Article filenames

In [6]:
vector_dict = [{},{},{}]
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()
print(len(crop_line_sep))
for idx,line in enumerate(crop_line_sep):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[0][word] = idx
pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()
print(len(pest_line_sep))
for idx,line in enumerate(pest_line_sep):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[1][word] = idx

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()
print(len(chem_line_sep))
for idx,line in enumerate(chem_line_sep):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[2][word] = idx
#Keywords split by lines, keyword with more than one entry will be on the same line
print(len(vector_dict[0]),len(vector_dict[1]),len(vector_dict[2]))

188
207
369
216 282 864


In [10]:
vectors = {}
#Vector table with filename as key and vector as value
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [[False]*188,[False]*207,[False]*369]
    #Initialize an 0 vector for each file
    for seg in seg_list:
        if(seg in vector_dict[0]):
            vectors[fname][0][vector_dict[0][seg]] = True
        if(seg in vector_dict[1]):
            vectors[fname][1][vector_dict[1][seg]] = True
        if(seg in vector_dict[2]):
            vectors[fname][2][vector_dict[2][seg]] = True
            #Count keyword appearence - Bag of words

for key in vectors:
    v = vectors[key]
    for i in range(3):
        curr = 0
        for b in v[i]:
            if(b):curr = (curr << 1) + 1
            else:curr = curr << 1
        vectors[key][i] = curr

[98079714615416886934934209737619787751599303819750539264, 51338600619007276755004625409535357651227760593150672896, 0]


In [45]:
def countSetBits(n):
    count = 0
    while (n):
        n &= (n-1)
        count+= 1
    return count

In [114]:
out = []
for Ref in txt_fnames:
    for Test in txt_fnames:
        if(Ref == Test):continue
        if((vectors[Test][0] == 0 and vectors[Test][1] == 0) or (vectors[Ref][0] == 0 and vectors[Ref][1] == 0)):continue
        if(vectors[Test][0] ^ (vectors[Test][0] & vectors[Ref][0]) == 0 and vectors[Test][1] ^ (vectors[Test][1] & vectors[Ref][1]) == 0 and vectors[Test][2] ^ (vectors[Test][2] & vectors[Ref][2]) == 0):
            out.append((Test,Ref))

In [115]:
corr_list = open('TrainLabel.csv', "r",encoding='UTF-8-sig')
corr = corr_list.read()
corr_line_sep = corr.splitlines()
pos_labels = set() #faster lookup to filter out positive pairs, not used elsewhere
#All the associated article pairs given by train label
for line in corr_line_sep[1:]:
    l = line.split(',')
    pos_labels.add((l[0],l[1]))

In [116]:
count = 0
for o in out:
    if(o in pos_labels):count+=1
print(count,len(out),len(pos_labels))

996 2205 1383


In [119]:
import csv
from natsort import natsorted
print(len(out))
out = natsorted(out,key = lambda x:x[0])
with open('val_super_simple_TEST.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Test"]+["Reference"])
    for row in out:  
        spamwriter.writerow(row)

2205


In [135]:
Ref = '415'
Test = '3'
#print(countSetBits(vectors[Test][0] ^ (vectors[Test][0] & vectors[Ref][0])),
#countSetBits(vectors[Test][1] ^ (vectors[Test][1] & vectors[Ref][1])),
#countSetBits(vectors[Test][2] ^ (vectors[Test][2] & vectors[Ref][2])))
txt = open(t_dir+'/'+Ref+'.txt', "r",encoding="utf-8")
content = txt.read()
print(content)
seg_list = jieba.cut(content, cut_all=True)
for seg in seg_list:
    if(seg in vector_dict[0] or seg in vector_dict[1]):print(seg)
print("----")
txt = open(t_dir+'/'+Test+'.txt', "r",encoding="utf-8")
content = txt.read()
print(content)
seg_list = jieba.cut(content, cut_all=True)
for seg in seg_list:
    if(seg in vector_dict[0] or seg in vector_dict[1]):print(seg)

夜蛾拉警報，白粉等著來，臺南區農改場籲請加強防治。臺南區農改場、防檢局及田邊好幫手關心您。
目前氣候乾燥及正值季節轉換期間，田間夜蛾類害蟲密度逐漸上升，尤以落花生田區發生嚴重。同時天氣漸轉冷涼，早晚溫差變大，白粉病發生機率提高，為確保作物正常生長及良好收成，臺南區農業改良場籲請農友注意夜蛾類害蟲及白粉病防治。常見危害夜蛾有斜紋夜蛾及甜菜夜蛾，兩者幼蟲皆晝伏夜出，雜食性，可危害作物如葉菜類、茼蒿、落花生、田菁、毛豆、豌豆、大蒜、青蔥、甘藷、玉米、蓮花及花卉等數十種作物。幼蟲食量極大，會大量啃食植株心梢及葉片，阻礙作物的正常生長，影響產量及品質，嚴重時常導致全園廢耕。白粉病是常見的作物葉部病害，感染初期通常由下位葉開始發生，葉表面出現白至淺灰之圓形小病斑，擴大癒合後即好似一層糖霜覆蓋整個葉面，影響光合作用，若遇久旱不雨的天氣，嚴重時葉片黃化，提早落葉。於春、秋兩季較乾燥及光線不足的環境下危害劇烈，由於白粉病亦會感染葉柄、嫩蔓等部位，如病害繼續進行，最終作物全株表面皆覆滿白色粉狀物而呈青白色。
夜蛾類害蟲防治上應注意下列事項：(1)田間如發現夜蛾類害蟲卵塊、幼蟲，應立即捕殺或摘除(2)幼蟲及成蟲大多於太陽下山或傍晚時分開始活動，傍晚噴藥效果較佳，噴灑時應注意使用充足水量(3)參考植物保護手冊推薦用藥，選擇藥劑交互使用，以避免害蟲產生抗藥性(4)利用性費洛蒙長期誘殺雄蛾，以降低其田間密度。每公頃設5~10個點，放置高度約為離地150公分或作物生長點上方30~60公分，每月更新誘餌一次。作物種植密集地區應採共同防治，成效更佳(5)注意清園之工作，間作之作物應同時施藥，徹底清除蟲源(6)旱田作物於種植前整地翻犁後浸水一天以上，可將土中的蛹或幼蟲淹死(7)收穫前注意化學藥劑的使用，或改用蘇力菌防治，注意安全採收期，以免造成農藥殘留過量之問題(8)受害嚴重之綠肥作物或廢耕田區應儘速翻犁淹水，避免夜蛾類滋生蔓延。白粉病防治法如下：(1)保持良好通風狀況，避免莖葉過於茂密。(2)發病初期可施用葵無露或窄域油。(3)參考作物登記藥劑，由於白粉病病原菌易產生抗藥性，因此應選擇多種藥劑，輪流施用，以避免抗藥性產生，並遵守安全採收期。白粉病葉面葉背都會發生，因此防治時應注意均勻噴施藥劑於葉片兩面。藥劑防治請參考植物保護手冊登記藥劑，可於本場網站連結各項作物的相關用藥資料：http://www.

In [170]:
test_pos = []
nons = 0
for Ref in vectors:
    for Test in vectors:
        if(Ref == Test):
            continue
        if(vectors[Ref][0] == 0 or vectors[Test][0] == 0):
            nons+=1
            continue
        if(Test == '1007' and Ref == '10'):print(((vectors[Ref][0] & vectors[Test][0]) == vectors[Test][0] and (vectors[Ref][1] & vectors[Test][1]) == vectors[Test][1] and (vectors[Ref][2] & vectors[Test][2]) == vectors[Test][2]))
        if((vectors[Ref][0] & vectors[Test][0]) == vectors[Test][0] and (vectors[Ref][1] & vectors[Test][1]) == vectors[Test][1] and (vectors[Ref][2] & vectors[Test][2]) == vectors[Test][2]):
            test_pos.append((Test,Ref))
bads = []
for t in test_pos:
    if(t not in pos_labels):bads.append(t)
print(len(bads))

True
1125


In [171]:
bads

[('1007', '10'),
 ('1015', '10'),
 ('893', '10'),
 ('988', '10'),
 ('892', '1016'),
 ('897', '1016'),
 ('898', '1016'),
 ('899', '1016'),
 ('961', '1016'),
 ('988', '1016'),
 ('35', '1025'),
 ('66', '1025'),
 ('1007', '103'),
 ('1015', '103'),
 ('893', '103'),
 ('988', '103'),
 ('897', '1037'),
 ('898', '1037'),
 ('1007', '104'),
 ('1015', '104'),
 ('893', '104'),
 ('988', '104'),
 ('1010', '1041'),
 ('289', '1041'),
 ('988', '1041'),
 ('1007', '105'),
 ('1015', '105'),
 ('1369', '105'),
 ('1373', '105'),
 ('1382', '105'),
 ('598', '105'),
 ('893', '105'),
 ('988', '105'),
 ('1357', '1056'),
 ('1381', '1056'),
 ('362', '1056'),
 ('1007', '1067'),
 ('1015', '1067'),
 ('893', '1067'),
 ('988', '1067'),
 ('1007', '1074'),
 ('1015', '1074'),
 ('893', '1074'),
 ('988', '1074'),
 ('1010', '1075'),
 ('1041', '1075'),
 ('289', '1075'),
 ('817', '1075'),
 ('988', '1075'),
 ('1007', '1076'),
 ('1015', '1076'),
 ('235', '1076'),
 ('893', '1076'),
 ('988', '1076'),
 ('1007', '1086'),
 ('1015', '10

In [189]:
Test = '899'
Ref = '1016'
txt = open(t_dir+'/'+Test+'.txt', "r",encoding="utf-8")
content = txt.read()
print(content)
seg_list = jieba.cut(content, cut_all=True)
for seg in seg_list:
    if(seg in vector_dict[0] or seg in vector_dict[1] or seg in vector_dict[2]):print(seg)
print("----")
txt = open(t_dir+'/'+Ref+'.txt', "r",encoding="utf-8")
content = txt.read()
print(content)
seg_list = jieba.cut(content, cut_all=True)
for seg in seg_list:
    if(seg in vector_dict[0] or seg in vector_dict[1] or seg in vector_dict[2]):print(seg)

屏東縣枋寮鄉5月上旬監測果實蠅密度達紅燈警戒標準，請農民加強防治工作，以降低田間蟲害密度。屏東縣政府、防檢局及田邊好幫手關心您。
依據屏東縣5月上旬植物疫情監測結果，屏東縣枋寮鄉監測果實蠅超過1000隻，密度達紅燈警戒標準，且持續上升，呼籲農友加強防治工作，以降低田間蟲害密度。
因果實蠅數量已達紅燈警戒標準，請各鄉鎮農會協助宣導農民加強防治工作，將果實套袋、清理果園中落果及懸掛含毒甲基丁香油誘餌，保持田間衛生清潔，才能有效壓制果實蠅族群發展，發揮防治功效。防治藥劑請參考防檢局農藥資訊服務網及植物保護資訊系統，並注意施用藥劑之安全採收期。

果實蠅
果實蠅
果實蠅
含毒甲基丁香油
毒甲基丁香油
甲基丁香油
丁香
丁香油
果實蠅
----
莫拉克颱風災後農作物復育措施
莫拉克颱風造成的淹水情況，使農田環境濕度提高，加上天氣放晴後溫度升高，易使農作物之疫病害蟲孳生蔓延，防檢局已立即加強各地農作物疫情監測工作，請農友亦特別注意田間排水及衛生管理，並留意疫病蟲害的發生與防治，以確保農作物生產安全。防檢局表示，本次颱風所帶來的強風豪雨，使二期作水稻發生稻熱病、紋枯病及白葉枯病之風險大為提高，因此請稻農應注意做好及時防治；另為預防蔬菜細菌性軟腐病、疫病、露菌病及黑腐病之危害，目前已受害嚴重之菜園或雜糧田，應立即進行清園，同時配合施藥防治，以有效抑制該等病害及斜紋夜蛾等害蟲之發生。防檢局指出，颱風造成的果樹枝、葉及果實表皮受傷，易導致病害感染蔓延為害，因此在天氣放晴後，應立即進行病害的防治。受害倒伏之果樹切忌勉強扶正，應於地上部先立柱固定，再視損傷程度適當修剪；颱風造成的落果，應即清除；已結果之果園其果實可採套袋或懸掛含毒甲基丁香油誘殺板，以預防病害的發生及避免果實蠅危害。防檢局再次呼籲農友，災後農作物復育工作務必先暢通排水，以利根部恢復生育，維持作物正常生長，受損嚴重田區，應先清園後復耕，並加強疫病蟲害防治，避免疫情擴大。農友如有作物疫病蟲害的問題，可就近聯繫農委會所屬試驗改良場所，或撥打該局植物病蟲害診斷服務專線：0800-069-880，向專家免費洽詢。


疫病
疫病
水稻
稻熱病
紋枯病
白葉枯病
蔬菜
軟腐病
疫病
露菌病
黑腐病
雜糧
斜紋夜蛾
果樹
果樹
含毒甲基丁香油
毒甲基丁香油
甲基丁香油
丁香
丁香油
果實蠅
疫病
疫病


(True, True, True)

('1007', '10')